In [1]:
import openmc
import openmc.deplete
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi
import xml.etree.ElementTree as et
import os

hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


In [2]:
trisoPkFrac = 0.3
pebblePkFrac = 0.64


#erchPercent = range(5,21);
erchPercent = [5, 5];
k_eff_results = []
U_form = 1;
O_form = 2;
rho_fuel = 10.96; # g/cm3

trisoPartThicknesses = [253.5, 97.7, 41.9, 37.5, 45.6]; #um

coolTemp = 450+273.15 #C -> K
coolPres = 8 #MPa

In [3]:
#Enrichment Optimization
#k_inf_erchPercent = range(len(erchPercent))

#      Buffer
buffer = openmc.Material(name='Buffer')
buffer.set_density('g/cm3', 1.0)
buffer.add_element('C', 1.0)
#buffer.add_s_alpha_beta('c_Graphite')

#      IPyC
IPyC = openmc.Material(name='IPyC')
IPyC.set_density('g/cm3', 1.9)
IPyC.add_element('C', 1.0)
#IPyC.add_s_alpha_beta('c_Graphite')

#      SiC
SiC = openmc.Material(name='SiC')
SiC.set_density('g/cm3', 3.2)
SiC.add_element('C', 0.5)
SiC.add_element('Si', 0.5)

#      OPyC
OPyC = openmc.Material(name='OPyC')
OPyC.set_density('g/cm3', 1.87)
OPyC.add_element('C', 1.0)
#OPyC.add_s_alpha_beta('c_Graphite')

# Moderator
graphite = openmc.Material()
graphite.set_density('g/cm3', 1.1995)
graphite.add_element('C', 1.0)
#graphite.add_s_alpha_beta('c_Graphite')

# Coolant
He = openmc.Material(name='He')
He.set_density('g/cm3',coolPres/coolTemp/2.0771)
He.add_element('He',1.0)
He.temperature = coolTemp

#B5
trisoPartRads = trisoPartThicknesses
for i in range(1,5):
    trisoPartRads[i] += trisoPartRads[i-1]
trisoPartVolFracs = trisoPartRads
for i in range(5):
    trisoPartVolFracs[i] = (trisoPartRads[i]/trisoPartRads[4])**3
for i in range(4):
    trisoPartVolFracs[4-i] -= trisoPartVolFracs[3-i]
pebbleRads = [25, 30]
#pebbleVol = 4/3*Math.pi*pebbleRads(1)^3
pebbleVolFracs = pebbleRads
for i in range(2):
    pebbleVolFracs[i] = (pebbleRads[i]/pebbleRads[1])**3
pebbleVolFracs[1] -= pebbleVolFracs[0]

for epi in erchPercent:
    #B4
    # TRISO Particles
    
    #      Fuel Kernel
    fuel = openmc.Material(name="fuel")
    fuel.set_density('g/cm3', rho_fuel)
    fuel.add_nuclide('U235',epi/100*U_form/(U_form+O_form))
    fuel.add_nuclide('U238',((1-epi/100)*U_form/(U_form+O_form)))
    fuel.add_element('O', 2.0)
    
    
    
    
    homogTrisoPart = openmc.Material.mix_materials([fuel, buffer, IPyC, SiC, OPyC], trisoPartVolFracs, 'vo')

    

    #B6
    
    homogTrisoPebCore = openmc.Material.mix_materials([homogTrisoPart,graphite], [trisoPkFrac, 1-trisoPkFrac], 'vo');
    homogTrisoPebble = openmc.Material.mix_materials([homogTrisoPebCore, graphite], pebbleVolFracs, 'vo');


    #B7
    homogCore = openmc.Material.mix_materials([homogTrisoPebble, He], [pebblePkFrac, 1-pebblePkFrac], 'vo')
    homogCore.add_s_alpha_beta('c_Graphite')
    materials = openmc.Materials()
    materials += [homogCore]
    materials.export_to_xml()


    #B8
    space = openmc.model.RectangularParallelepiped(-15, 15, -15, 15, -15, 15, boundary_type = 'reflective')
    rxr = openmc.Cell(fill=homogCore, region=-space)
    universe = openmc.Universe(cells=[rxr])
    geometry = openmc.Geometry()
    geometry.root_universe = universe
    geometry.export_to_xml()
    settings = openmc.Settings()
    settings.run_mode = 'eigenvalue'
    settings.particles = 25000
    settings.batches = 45
    settings.inactive = 15
    settings.temperature={'method':'interpolation','range':(250,2500)}
    
    # add a fission source
    box = openmc.stats.Box(lower_left=(-15,-15,-15),
                          upper_right =(15,15,15),only_fissionable=True)
    src = openmc.Source(space=box)
    settings.source = src    
    
    # now export to xml
    settings.export_to_xml()


    #B9
    num_threads_des = 36  # set as desired.  Can be useful to ensure you do not take over all the resources for a machine
    num_threads = min(os.cpu_count(),num_threads_des); # prevent asking for more threads than OMP_NUM_THREADS
    
    openmc.run(threads = num_threads, output=True)
    
    sp = openmc.StatePoint('statepoint.45.h5')
    keff = sp.keff;
    k_eff_results.append(keff.nominal_value)
    sp.close()


    #B10
    #ktup = openmc.lib.keff()
    #epi = ktup[0]
    #print(epi)
    #k_eff_results.append(ktup[0]) 

hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                #########

 Reading O16 from /home/sblair/OMC_DATA/endfb71_hdf5/O16.h5
 Reading O17 from /home/sblair/OMC_DATA/endfb71_hdf5/O17.h5
 Reading C0 from /home/sblair/OMC_DATA/endfb71_hdf5/C0.h5
 Reading Si28 from /home/sblair/OMC_DATA/endfb71_hdf5/Si28.h5
 Reading Si29 from /home/sblair/OMC_DATA/endfb71_hdf5/Si29.h5
 Reading Si30 from /home/sblair/OMC_DATA/endfb71_hdf5/Si30.h5
 Reading He3 from /home/sblair/OMC_DATA/endfb71_hdf5/He3.h5
 Reading He4 from /home/sblair/OMC_DATA/endfb71_hdf5/He4.h5
 Reading c_Graphite from /home/sblair/OMC_DATA/endfb71_hdf5/c_Graphite.h5
 Minimum neutron data temperature: 250 K
 Maximum neutron data temperature: 2500 K
 Preparing distributed cell instances...
 Reading plot XML file...
 Writing summary.h5 file...
 Maximum neutron transport energy: 20000000 eV for U235
 Initializing source particles...

 ====================>     K EIGENVALUE SIMULATION     <====================

  Bat./Gen.      k            Average k
  =========   ========   ====================
        1

In [4]:
print(k_eff_results)

[0.9931093440119736, 0.9931093440119632]


In [5]:
#print(k_inf_erchPercent)